In [24]:
# regression_anomaly_detection.py

# Importing necessary libraries for regression and anomaly detection
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import IsolationForest
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder


In [25]:
# Load the preprocessed data
filepath = 'datasets/Housing_v2.csv' 
data = pd.read_csv(filepath)

In [26]:
data = data.dropna()

In [27]:
# Identify numerical and non-numerical columns
numerical_columns = data.select_dtypes(include=['number']).columns.tolist()
non_numerical_columns = data.select_dtypes(exclude=['number']).columns.tolist()

X = data[[
#    'Description',
   'Location',
   #'Price',
   'Bedrooms',
   'Bathrooms',
   'Floor Area',
   'Land Area',
   #'Latitude',
   #'Longitude'
]] 

y = data['Price']  # Target variable



In [28]:
# Identify numerical and non-numerical columns
numerical_columns = data.select_dtypes(include=['number']).columns.tolist()
non_numerical_columns = data.select_dtypes(exclude=['number']).columns.tolist()

# Label encode the non-numeric features
label_encoder = LabelEncoder()

# Apply label encoding to non-numerical columns
for column in non_numerical_columns:
    if column in X.columns:  # Check if the column exists in the feature set
        X[column] = label_encoder.fit_transform(X[column])


C:\Users\fabia\AppData\Local\Temp\ipykernel_23860\1518733174.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])


In [29]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Display the shapes of the splits
X.shape, X_train.shape, X_test.shape

((1055, 5), (844, 5), (211, 5))

### Random Forest


In [30]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
import numpy as np

# Define and fit the model with default or basic custom settings
model = GradientBoostingRegressor(
    n_estimators=100,         # Default number of trees
    learning_rate=0.1,        # Default learning rate
    max_depth=3,              # Default depth
    random_state=42           # Ensure reproducibility
)

# Train the model
model.fit(X_train, y_train)

# Make predictions
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

# Train evaluation metrics
train_mae = mean_absolute_error(y_train, y_train_pred)
train_mse = mean_squared_error(y_train, y_train_pred)
train_rmse = np.sqrt(train_mse)
train_r2 = r2_score(y_train, y_train_pred)
train_mape = mean_absolute_percentage_error(y_train, y_train_pred)

# Test evaluation metrics
test_mae = mean_absolute_error(y_test, y_test_pred)
test_mse = mean_squared_error(y_test, y_test_pred)
test_rmse = np.sqrt(test_mse)
test_r2 = r2_score(y_test, y_test_pred)
test_mape = mean_absolute_percentage_error(y_test, y_test_pred)

# Print metrics
print("\nTrain Metrics:")
print(f"MAE: {train_mae}")
print(f"MSE: {train_mse}")
print(f"RMSE: {train_rmse}")
print(f"R²: {train_r2}")
print(f"MAPE: {train_mape}")

print("\nTest Metrics:")
print(f"MAE: {test_mae}")
print(f"MSE: {test_mse}")
print(f"RMSE: {test_rmse}")
print(f"R²: {test_r2}")
print(f"MAPE: {test_mape}")



Train Metrics:
MAE: 7116732.0960393
MSE: 238961516702648.2
RMSE: 15458380.14484856
R²: 0.9705703947185024
MAPE: 0.3792420848476742

Test Metrics:
MAE: 10483973.214681957
MSE: 593828598929934.5
RMSE: 24368598.624663144
R²: 0.8899353345532601
MAPE: 0.4142765076311706


In [31]:
# Sample data: predicted and actual prices
predicted_prices = model.predict(X_test)  # Get the predicted prices from the model
actual_prices = y_test  # Actual prices from the test set

# Create a DataFrame to compare predicted vs actual prices
comparison_df = pd.DataFrame({
    'Predicted Price': predicted_prices,
    'Actual Price': actual_prices
})

# Optionally, calculate the error (difference between predicted and actual prices)
comparison_df['Error'] = comparison_df['Predicted Price'] - comparison_df['Actual Price']

# Print the first few rows of the comparison
print(comparison_df.head())

# Optionally, save it to a CSV for further analysis
comparison_df.to_csv('predicted_vs_actual_prices.csv', index=False)


      Predicted Price  Actual Price         Error
337      4.833130e+06     4079000.0  7.541297e+05
1125     1.852134e+07    12000000.0  6.521338e+06
1148     3.806924e+08   460000000.0 -7.930759e+07
1416     5.084078e+06     3500000.0  1.584078e+06
120      4.819979e+06     3970000.0  8.499794e+05


In [32]:
# # Visualizing the predicted vs actual prices
# plt.figure(figsize=(10, 6))
# plt.scatter(y_test, y_pred, color='blue')
# plt.plot([min(y_test), max(y_test)], [min(y_test), max(y_test)], color='red', linewidth=2)
# plt.title('Predicted vs Actual Prices')
# plt.xlabel('Actual Prices')
# plt.ylabel('Predicted Prices')
# plt.show()


In [33]:
# # Anomaly detection using Isolation Forest
# iso_forest = IsolationForest(contamination=0.05, random_state=42)
# outliers = iso_forest.fit_predict(X)

# # Marking anomalies as -1 and normal points as 1
# data['anomaly'] = outliers

# # Display rows with anomalies
# anomalies = data[data['anomaly'] == -1]
# anomalies.head()


In [34]:
# # Visualizing anomalies in the price vs square footage scatter plot
# plt.figure(figsize=(10, 6))
# sns.scatterplot(x=data['square_footage'], y=data['price'], hue=data['anomaly'], palette='coolwarm', legend=False)
# plt.title('Price vs Square Footage with Anomalies')
# plt.xlabel('Square Footage')
# plt.ylabel('Price')
# plt.show()
